In [102]:
#!/usr/bin/env python
"""
CEVAE moargs.latent_dimensionel on IHargs.latent_dimensionP
"""
##################### Neeargs.latent_dimension TF1
# import edward as ed
# import tensorflow as tf
# from utils import fc_net, get_y0_y1
# from edward.models import Bernoulli, Normal
# from progressbar import ETA, Bar, Percentage, ProgressBar
###############################################################
import os
import json
import time
import math
import numpy as np
import pandas as pd
import tensorflow as tf
tf.enable_v2_behavior()
from scipy.stats import sem
import tensorflow.keras as tfk
import tensorflow_probability as tfp
import tensorflow.keras.layers as tfkl
tfd,tfpl = tfp.distributions,tfp.layers
import tensorflow.keras.backend as tfkb
from sklearn.metrics import roc_auc_score

from datasets import IHDP
from evaluation import Evaluator
from argparse import ArgumentParser

parser = ArgumentParser()
################### CEVAE
parser.add_argument('-reps', type=int, default=10)
parser.add_argument('-earl', type=int, default=10)
parser.add_argument('-lr', type=float, default=0.001)
parser.add_argument('-opt', choices=['adam', 'adamax'], default='adam')
parser.add_argument('-epochs', type=int, default=100)
parser.add_argument('-print_every', type=int, default=10)
################### Reckon Need
parser.add_argument('--model_path',  type = str, default='./',help = 'Model save path.')
parser.add_argument('--train_paths', type = str, default='',  help = 'HDFS paths to input train files.')
parser.add_argument('--test_paths',  type = str, default='',  help = 'HDFS paths to input test files.')
################### Data Path Self
parser.add_argument('--train_data_paths',  type = str, 
                    default="hdfs://haruna/user/yanyiming.work/coupon/new_user/rp_label_30day_all_convert_split_v1.1/tfrecord/train")
parser.add_argument('--test_data_paths',   type = str, 
                    default="hdfs://haruna/user/yanyiming.work/coupon/new_user/rp_label_30day_all_convert_split_v1.1/tfrecord/test")
parser.add_argument('--date_interval',     type = str, default="20211114,20211124")
################### Load Model Path
parser.add_argument('--last_model_path',   type = str, default='', help='last model path.')
parser.add_argument('--load_latest_model', type = str, default='', help='load from the latest dump of this model')
################### Treatment And Label
parser.add_argument('--label', type = str, default='rpg30', help = '')
parser.add_argument('--target_treat',  type = str, default = 'threshold_cents', help = 'target treat')
parser.add_argument('--treatment_col', type = str, default = 'credit_cents,threshold_cents', help = 'all treatment')
parser.add_argument('--control_treatment_val', type = float, default = 0.0,   help = 'control id')
parser.add_argument('--experim_treatment_val', type = float, default = 800.0, help = 'experiment id')
################### BASIC
parser.add_argument('--num_parallel_reads', type = int, default = 10, help = 'random seed')
parser.add_argument('--random_seed', type = int,default = 9418, help = 'random seed')
parser.add_argument('--mode', type = str, default = 'train', help = 'train or evaluate')
args = parser.parse_args([])
args.true_post = True

#################################From CEVAE
dataset = IHDP(replications=args.reps)
dimx = 25
scores = np.zeros((args.reps, 3))
scores_test = np.zeros((args.reps, 3))

#################################IHDP Data

################################# some functions needed

def findfile(start, name):
    ans = []
    for relpath, dirs, files in os.walk(start):
        for file in files:
            if name in file:
                full_path = os.path.join(start, file)
                ans.append(full_path)
                # print(os.path.normpath(os.path.abspath(full_path)))
    return ans

class ExpandDims(tf.keras.layers.Layer):
    def __init__(self, axis=1, **kwargs):
        super(ExpandDims, self).__init__(**kwargs)
        self.axis = axis

    def build(self, input_shape):
        super(ExpandDims, self).build(input_shape)

    def call(self, input):
        out = tf.expand_dims(input, axis=self.axis)
        return out

    def compute_output_shape(self, input_shape):
        return [input_shape[0], 1]

# for data in input_fn_csv("./datasets/IHDP/csv/",".csv",batch_size=2):
#     break
# data['features'],data['labels']

# data information 
t_bin_dim = 1
y_dim, default_y_scale = 1,tf.exp(0.)
M = None        # batch size during training
z_dim = 20          # latent z dimension
lamba = 1e-4    # weight decay
nh, h = 3, 200  # number and size of hidden layers
binfeats = [6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
numfeats = [i for i in range(25) if i not in binfeats]
x_bin_dim = len(binfeats)
x_num_dim = len(numfeats)


In [105]:
def ihdp_fn(data):
    data = tf.strings.to_number(tf.strings.split(data, sep=','))
    t, y, y_cf = data[0:1], data[1:2], data[2:3]
    mu_0, mu_1, x = data[3:4], data[4:5], data[5:]
    # this binary feature is in {1, 2}
    tmp = [0 for i in range(25)]
    tmp[13] = 1
    # print(x,t,y)
    return [
        tf.concat([x,t,y], axis=-1),
        tf.concat([y_cf, mu_0, mu_1], axis=-1)
        ]
def input_fn_csv(path,name, batch_size=128, shuffle_size=0):
    all_files = findfile(path,name)
    ds = tf.data.TextLineDataset(
        all_files,
        compression_type= None,
        num_parallel_reads=int(args.num_parallel_reads),
        buffer_size=64*1024*1024# 可选参数，多线程并行读多个文件
    ).repeat(1)
    if shuffle_size > 0:
        ds = ds.shuffle(shuffle_size)
    ds = ds.map(ihdp_fn).batch(batch_size)
    return ds

class CEVAE(tf.keras.Model):
    def __init__(self):
        super(CEVAE, self).__init__()
        ########################################
        self.activation = 'elu'
        # CEVAE Model (decoder)
        # p(z)
        self.z = tfd.Normal(loc = [0]*z_dim, scale = [1]*z_dim)
        # p(x|z)
        self.p_x_z_shared = self.fc_net(z_dim,(nh - 1) * [h], [], 'px_z_shared', lamba=lamba, activation=self.activation)
        self.p_x_z_bin = tfk.Sequential(
            [tfkl.InputLayer(input_shape=[h]),
            tfkl.Dense(h,activation=self.activation),
            tfkl.Dense(tfpl.IndependentBernoulli.params_size(x_bin_dim),activation=None),
            tfkl.Flatten(),
            tfpl.IndependentBernoulli(x_bin_dim,tfd.Bernoulli.logits)
            ])
        
        self.p_x_z_num = tfk.Sequential(
            [tfkl.InputLayer(input_shape=[h]),
            tfkl.Dense(h,activation=self.activation),
            tfkl.Flatten(),
            tfkl.Dense(tfpl.MultivariateNormalTriL.params_size(z_dim),
                       activation=None),
            tfpl.MultivariateNormalTriL(
                z_dim,
                # activity_regularizer=tfpl.KLDivergenceRegularizer(self.z)
                ),
        ])

            
            
        # q(z|x,t,y)
        self.q_z_x = tfk.Sequential(
            [self.fc_net(x_bin_dim + x_num_dim,(nh-1) * [h], [], 'qz_x_shared', lamba=lamba, activation=self.activation),
            tfkl.Dense(h,activation=self.activation),
            tfkl.Dense(tfpl.IndependentNormal.params_size(z_dim),activation=None),
            tfkl.Flatten(),
            tfpl.IndependentNormal(z_dim)
            ])

    def fc_net(self,input_shape, layers, out_layers, scope, lamba=1e-3, activation=tf.nn.relu):
        net = tfk.Sequential([tfkl.InputLayer([input_shape])])
        for hidden in layers:
            net.add(tfkl.Dense(hidden, activation = self.activation))
        if out_layers:
            [outdim, activation] = out_layers
            net.add(tfkl.Dense(outdim, activation = activation))
        return net

    # @tf.function
    def call(self, features, training=False, serving=False):
        # input data preprocess
        x_ph_bin = features[...,0:x_bin_dim]  # binary inputs
        x_ph_num = features[...,x_bin_dim:x_bin_dim+x_num_dim]  # continuous inputs

        x_ph = tf.concat([x_ph_bin, x_ph_num], -1)
        # p(z)
        z = self.z.sample(tf.shape(x_ph)[0])
        # print("**** z ****",z)

        # CEVAE variational approximation (encoder)
        # q(z|x)
        q_z_x_dist = self.q_z_x(x_ph)
        # print("**** qz ****",q_z_x_dist)

        # CEVAE model (decoder)
        # p(x|z)
        hx = self.p_x_z_shared(q_z_x_dist)
        x_bin = self.p_x_z_bin(hx)
        x_num = self.p_x_z_num(hx)

        output = x_bin
        # print("**** output ****", output)
        return output

    # @tf.function
    # def serve(self, input):
    #     return self(input, serving=True)

def cevae_loss(labels,preds):
    print("**** loss output ****", preds)
    print("cevae loss")

    # read data
    x_bin_real = labels[..., 0:x_bin_dim]  # binary inputs
    x_num_real = labels[..., (x_bin_dim):(x_bin_dim+x_num_dim)]  # continuous inputs
    t_real     = labels[..., (x_bin_dim+x_num_dim):(x_bin_dim+x_num_dim+t_bin_dim)]
    y_real     = labels[..., (x_bin_dim+x_num_dim+t_bin_dim):(x_bin_dim+x_num_dim+t_bin_dim+y_dim)]
    # print("**** t_real ****",t_real)

    return 1


def build_model():
    model = CEVAE()
    features = tf.keras.Input(x_bin_dim + x_num_dim + t_bin_dim + y_dim,dtype=tf.float32)
    fake_inputs = features
    model(fake_inputs)
    model.compile(optimizer=tf.keras.optimizers.Adam(args.lr),
                  loss= cevae_loss,
                  metrics=[])
    return model

In [106]:
for i in input_fn_csv("./datasets/IHDP/csv",'.csv'):
    break
model = build_model()
model.fit(
    x = i[0],
    y = i[0],
    epochs = 5
    )

Epoch 1/5
**** loss output **** tfp.distributions._TensorCoercible("tensor_coercible", batch_shape=[32], event_shape=[19], dtype=float32)
cevae loss


ValueError: in user code:

    File "/usr/local/lib/python3.9/site-packages/keras/engine/training.py", line 878, in train_function  *
        return step_function(self, iterator)
    File "/usr/local/lib/python3.9/site-packages/keras/engine/training.py", line 867, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/usr/local/lib/python3.9/site-packages/keras/engine/training.py", line 860, in run_step  **
        outputs = model.train_step(data)
    File "/usr/local/lib/python3.9/site-packages/keras/engine/training.py", line 816, in train_step
        self.optimizer.minimize(loss, self.trainable_variables, tape=tape)
    File "/usr/local/lib/python3.9/site-packages/keras/optimizer_v2/optimizer_v2.py", line 532, in minimize
        return self.apply_gradients(grads_and_vars, name=name)
    File "/usr/local/lib/python3.9/site-packages/keras/optimizer_v2/optimizer_v2.py", line 633, in apply_gradients
        grads_and_vars = optimizer_uti